In [1]:
import sys
from datetime import datetime, timedelta
import requests
import fileinput
import os.path
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings("ignore")

from pandas.plotting import autocorrelation_plot
from matplotlib import pyplot
import seaborn as sns

Oht_enc = OneHotEncoder()
today = datetime.today() - timedelta(days=2)


In [2]:
fileToSearch  = 'https://dadosabertos.c3sl.ufpr.br/curitiba/CliqueEconomia/'+str(today.date())+'_Clique_Economia_-_Base_de_Dados.csv'

textToSearch1 =  ", ,250"
textToReplace1 = ",250" 
textToSearch2 = ",,18"
textToReplace2 =  ",18" 
textToSearch3 = "/"
textToReplace3 =  "-"

print(fileToSearch)
response = requests.get(fileToSearch)
open('working/origem.csv', 'wb').write(response.content)

fileOrigem  = 'working/origem.csv'
fileOk  = 'working/dados.csv'
fileFinal  = 'working/final.csv'

with open(fileOrigem, encoding='utf-8', mode='r') as file:
    data = file.read()
    data = data.replace(textToSearch1, textToReplace1)
    data = data.replace(textToSearch2, textToReplace2)
    data = data.replace(textToSearch3, textToReplace3)

with open(fileOk, encoding='utf-8', mode='w') as file:
    file.write(data)


https://dadosabertos.c3sl.ufpr.br/curitiba/CliqueEconomia/2023-02-20_Clique_Economia_-_Base_de_Dados.csv


In [3]:
df = pd.read_csv(fileOk,sep=',')

df["data_pesquisa"]= pd.to_datetime(df["data_pesquisa"])

ufilial = df['id_filial'].unique()
uprod = df['id_produto'].unique()

removerNegativo = df[(df['preco_encontrado'] <= 0)]
if not removerNegativo.empty:
  df = df.drop(index=removerNegativo.index[0])

for f in ufilial:
    for p in uprod:
        dataf = df[(df['id_filial'] == f) & (df['id_produto'] == p)]
        if not dataf.empty:
          q1 = np.percentile(dataf['preco_encontrado'], 25,interpolation = 'midpoint')
          q3 = np.percentile(dataf['preco_encontrado'], 75,interpolation = 'midpoint')
          iqr = q3 - q1
          upper=q3+5*iqr
          lower=q1-5*iqr
          removerUpper = df[(df['id_filial'] == f) & (df['id_produto'] == p) & (df['preco_encontrado'] >= upper)]
          if not removerUpper.empty:
            df = df.drop(index=removerUpper.index[0])
          removerLower = df[(df['id_filial'] == f) & (df['id_produto'] == p) & (df['preco_encontrado'] <= lower)]
          if not removerLower.empty:
            df = df.drop(index=removerLower.index[0])
    

df.info()
df.head()

df.to_csv(fileFinal, sep=';', encoding='utf-8')